In [1]:
# === CONFIG GLOBAL ===
from pathlib import Path
import os

# Carpeta para resultados y descargas
DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
IMG_DIR  = Path("imgs"); IMG_DIR.mkdir(exist_ok=True)

# Bandas que usaremos (incluye todas las necesarias para NDCI/FAI y máscaras)
BANDS_FULL = ["B02","B03","B04","B05","B07","B8A","B11","B12"]

# AOIs
LAKES = {
    "Atitlán":   {"west": -91.349, "east": -91.0702, "south": 14.5971, "north": 14.7648},
    "Amatitlán": {"west": -90.66,  "east": -90.58,   "south": 14.43,   "north": 14.51},
}

CSV_PATH = DATA_DIR / "chl_timeseries.csv"

# Umbrales (ajustables)
MNDWI_THR = 0.42
NDWI_THR  = 0.40
FAI_FLOATING_THR = 0.08
CHL_CLIP_UPPER = 500.0   # para evitar explosiones en la media

In [3]:
import numpy as np, pandas as pd, rasterio, glob
import openeo


def connect_openeo():
    return openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()


def download_s2_tif(conn, aoi, dates, title, out_dir=IMG_DIR, cloud_cover=20):
    """
    Descarga un GeoTIFF multibanda para la ventana 'dates'.
    Usa filtro de nubosidad si se especifica.
    Devuelve la ruta del último .tif descargado.
    """
    kwargs = dict(
    collection_id="SENTINEL2_L2A",
    spatial_extent=aoi,
    temporal_extent=dates,
    bands=BANDS_FULL
    )

    cube = conn.load_collection(**kwargs)
    graph = cube.save_result(format="GTIFF")
    job = conn.create_job(graph, title=title)
    job.start_and_wait()
    out_dir.mkdir(exist_ok=True)
    job.download_results(str(out_dir))
    tifs = sorted(out_dir.glob("*.tif"), key=lambda p: p.stat().st_mtime)
    if not tifs:
        raise RuntimeError("No se descargó ningún .tif. Prueba con otra ventana de fechas o mayor tolerancia de nubes.")
    return str(tifs[-1])


def load_bands(tif_path):
    with rasterio.open(tif_path) as src:
        data = src.read()
        nodata = src.nodata
    B02,B03,B04,B05,B07,B8A,B11,B12 = [data[i].astype("float32") for i in range(8)]
    return dict(B02=B02, B03=B03, B04=B04, B05=B05, B07=B07, B8A=B8A, B11=B11, B12=B12), nodata


def to_reflectance(b, nodata=None, scale=10000.0):
    out = b.astype("float32")
    if nodata is not None:
        out = np.where(b == nodata, np.nan, out)
    return out / scale


def scale_all(bands, nodata=None):
    return {k: to_reflectance(v, nodata=nodata) for k,v in bands.items()}


def water_mask(R):
    B02,B03,B04,B8A,B11,B12 = R["B02"],R["B03"],R["B04"],R["B8A"],R["B11"],R["B12"]
    ndwi  = (B03 - B8A) / (B03 + B8A + 1e-9)
    mndwi = (B03 - B11) / (B03 + B11 + 1e-9)
    aweish  = B02 + 2.5*B03 - 1.5*(B8A + B11) - 0.25*B12
    aweinsh = 4*(B03 - B11) - (0.25*B8A + 2.75*B11)
    ndvi = (B8A - B04) / (B8A + B04 + 1e-9)
    dbsi = ((B11 - B03) / (B11 + B03 + 1e-9)) - ndvi
    ws = ( (mndwi > MNDWI_THR) | (ndwi > NDWI_THR) | (aweinsh > 0.1879) | (aweish > 0.1112) | (ndvi < -0.2) )
    ws = ws & ~((aweinsh <= -0.03) | (dbsi > 0))
    return ws


def FAI(B04, B07, B8A):
    return (B07 - (B04 + (B8A - B04) * (783 - 665) / (865 - 665)))


def NDCI(B04, B05):
    return (B05 - B04) / (B05 + B04 + 1e-9)


def chl_exp(ndci):
    # Clip NDCI a rango razonable para evitar overflow
    ndci = np.clip(ndci, -1.0, 1.0)
    return 17.441 * np.exp(4.7038 * ndci)


def chlorophyll_map(R, mask_water, use_exp=True, exclude_floating=True):
    B04,B05,B07,B8A = R["B04"], R["B05"], R["B07"], R["B8A"]
    fai  = FAI(B04, B07, B8A)
    ndci = NDCI(B04, B05)
    chl  = chl_exp(ndci) if use_exp else (826.57*(ndci**3) - 176.43*(ndci**2) + 19*ndci + 4.071)
    chl  = np.where(mask_water, chl, np.nan)
    if exclude_floating:
        chl = np.where(fai > FAI_FLOATING_THR, np.nan, chl)
    # Anti-outliers
    chl = np.where(np.isfinite(chl), chl, np.nan)
    chl = np.where(chl > CHL_CLIP_UPPER, np.nan, chl)
    return chl


def summarize_chl(chl):
    arr = chl[np.isfinite(chl)]
    if arr.size == 0:
        return dict(mean=np.nan, median=np.nan, p25=np.nan, p75=np.nan, n=0)
    return dict(
        mean=float(arr.mean()),
        median=float(np.median(arr)),
        p25=float(np.percentile(arr,25)),
        p75=float(np.percentile(arr,75)),
        n=int(arr.size)
    )


def append_timeseries(csv_path, new_rows_df):
    try:
        hist = pd.read_csv(csv_path, parse_dates=["date"])
        out = pd.concat([hist, new_rows_df], ignore_index=True)
    except FileNotFoundError:
        out = new_rows_df.copy()

    out = (out
           .drop_duplicates(subset=["lake","date"], keep="last")
           .sort_values(["lake","date"]))

    tmp = csv_path.with_suffix(".tmp.csv")
    out.to_csv(tmp, index=False, encoding="utf-8")
    os.replace(tmp, csv_path)  # escritura atómica
    return out


def clean_tiffs(folder=IMG_DIR, pattern="*.tif"):
    for p in Path(folder).glob(pattern):
        try:
            p.unlink()
        except Exception as e:
            print("No se pudo borrar", p, e)


def process_week_for_lake(conn, lake_name, aoi, date_window, scene_date, cloud_cover=20):
    """
    pipeline: descarga -> bandas -> reflectancia -> máscara -> clorofila -> resumen
    Devuelve dict con stats y también hace limpieza de TIFFs.
    """
    title = f"S2 {lake_name} all bands {date_window[0]}..{date_window[1]}"
    tif_path = download_s2_tif(conn, aoi, date_window, title=title, cloud_cover=cloud_cover)

    bands, nodata = load_bands(tif_path)
    R = scale_all(bands, nodata=nodata)
    water = water_mask(R)
    chl = chlorophyll_map(R, water, use_exp=True, exclude_floating=True)
    stats = summarize_chl(chl)

    # armar fila
    row = {
        "date": pd.to_datetime(scene_date),
        "lake": lake_name,
        "chl_mean":  stats["mean"],
        "chl_median":stats["median"],
        "chl_p25":   stats["p25"],
        "chl_p75":   stats["p75"],
    }

    # limpia para no llenar disco
    clean_tiffs()
    return row

In [4]:
import sys, platform
import numpy as np, pandas as pd, matplotlib
import rasterio, statsmodels

print("Python:", sys.version.split()[0])
print("OS:", platform.platform())
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("matplotlib:", matplotlib.__version__)
print("rasterio:", rasterio.__version__)
import statsmodels.api as sm
print("statsmodels:", sm.__version__)


Python: 3.10.2
OS: macOS-15.5-arm64-arm-64bit
numpy: 2.2.6
pandas: 2.3.1
matplotlib: 3.10.5
rasterio: 1.4.3
statsmodels: 0.14.5


In [8]:
#!pip install -q openeo rasterio numpy pandas matplotlib

In [5]:
# === CAMBIA ESTAS DOS LINEAS CADA SEMANA ===
fechas_semana_atitlan   = ["2025-02-11", "2025-02-15"]  # ventana corta
fecha_escena_atitlan    = "2025-02-13"                  # fecha real elegida

fechas_semana_amatitlan = ["2025-02-16", "2025-02-20"]
fecha_escena_amatitlan  = "2025-02-18"

# -------------------------------------------
conn = connect_openeo()

rows = []
rows.append(
    process_week_for_lake(conn, "Atitlán",   LAKES["Atitlán"],   fechas_semana_atitlan,   fecha_escena_atitlan,   cloud_cover=20)
)
rows.append(
    process_week_for_lake(conn, "Amatitlán", LAKES["Amatitlán"], fechas_semana_amatitlan, fecha_escena_amatitlan, cloud_cover=20)
)

df_new = pd.DataFrame(rows)
df_out = append_timeseries(CSV_PATH, df_new)
print("Actualizado:", CSV_PATH, "→ total filas:", len(df_out))
display(df_new)

Authenticated using refresh token.
0:00:00 Job 'j-25081420322348e9a1de4b1e161dfc79': send 'start'
0:00:13 Job 'j-25081420322348e9a1de4b1e161dfc79': created (progress 0%)
0:00:19 Job 'j-25081420322348e9a1de4b1e161dfc79': created (progress 0%)
0:00:25 Job 'j-25081420322348e9a1de4b1e161dfc79': created (progress 0%)
0:00:34 Job 'j-25081420322348e9a1de4b1e161dfc79': created (progress 0%)
0:00:44 Job 'j-25081420322348e9a1de4b1e161dfc79': created (progress 0%)
0:00:56 Job 'j-25081420322348e9a1de4b1e161dfc79': running (progress N/A)
0:01:12 Job 'j-25081420322348e9a1de4b1e161dfc79': running (progress N/A)
0:01:32 Job 'j-25081420322348e9a1de4b1e161dfc79': running (progress N/A)
0:01:56 Job 'j-25081420322348e9a1de4b1e161dfc79': finished (progress 100%)


/var/folders/kd/4y1c0b3j1273_pm_gb0hfhdw0000gn/T/ipykernel_25750/2224465159.py:27: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(str(out_dir))
/Users/gerco/UVG/8th_semester/Data_Science/DC-Lab4/venv/lib/python3.10/site-packages/openeo/rest/job.py:199: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
/Users/gerco/UVG/8th_semester/Data_Science/DC-Lab4/venv/lib/python3.10/site-packages/openeo/rest/job.py:203: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


0:00:00 Job 'j-2508142036274e9590e07ebf560254f2': send 'start'
0:00:13 Job 'j-2508142036274e9590e07ebf560254f2': created (progress 0%)
0:00:19 Job 'j-2508142036274e9590e07ebf560254f2': created (progress 0%)
0:00:25 Job 'j-2508142036274e9590e07ebf560254f2': created (progress 0%)
0:00:33 Job 'j-2508142036274e9590e07ebf560254f2': created (progress 0%)
0:00:44 Job 'j-2508142036274e9590e07ebf560254f2': running (progress N/A)
0:00:56 Job 'j-2508142036274e9590e07ebf560254f2': running (progress N/A)
0:01:12 Job 'j-2508142036274e9590e07ebf560254f2': running (progress N/A)
0:01:31 Job 'j-2508142036274e9590e07ebf560254f2': running (progress N/A)
0:01:55 Job 'j-2508142036274e9590e07ebf560254f2': finished (progress 100%)
Actualizado: data/chl_timeseries.csv → total filas: 2


,date,lake,chl_mean,chl_median,chl_p25,chl_p75
0,2025-02-13,Atitlán,15.209627,14.985576,12.875841,17.82654
1,2025-02-18,Amatitlán,21.114990,21.114395,20.754000,21.48053


In [6]:
# Entrenamiento rápido ARIMA cuando existan suficientes puntos
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

try:
    ts = pd.read_csv(CSV_PATH, parse_dates=["date"])
except FileNotFoundError:
    print("Aún no existe el CSV", CSV_PATH)
else:
    ORDER = (1,1,1)
    STEPS = 4
    MIN_PTS = 8

    for lake in ts["lake"].unique():
        df_lake = (ts[ts["lake"]==lake]
                   .sort_values("date")
                   .set_index("date"))

        # semanal: por si las fechas no caen en el mismo día
        y = df_lake["chl_mean"].resample("7D").mean().interpolate(limit_direction="both")
        y = y.mask((~np.isfinite(y)) | (y <= 0)).dropna()

        if len(y) < MIN_PTS:
            print(f"[{lake}] pocos puntos ({len(y)}). Sigue agregando semanas.")
            continue

        res = ARIMA(y, order=ORDER).fit()
        print(f"[{lake}] AIC={res.aic:.2f} puntos={len(y)}")

        fc = res.get_forecast(steps=STEPS)
        mean_fc = fc.predicted_mean
        ci = fc.conf_int()
        future_idx = pd.date_range(start=y.index[-1] + pd.Timedelta(days=7), periods=STEPS, freq="7D")

        plt.figure(figsize=(8,4))
        plt.plot(y.index, y.values, marker="o", label="Observado")
        plt.plot(future_idx, mean_fc.values, marker="x", label="Pronóstico")
        plt.fill_between(future_idx, ci.iloc[:,0], ci.iloc[:,1], alpha=0.2)
        plt.title(f"Chl-a media · {lake}")
        plt.xlabel("Fecha"); plt.ylabel("mg/m³ aprox.")
        plt.grid(True, alpha=0.3); plt.legend()
        plt.show()

[Amatitlán] pocos puntos (1). Sigue agregando semanas.
[Atitlán] pocos puntos (1). Sigue agregando semanas.


In [7]:
# === PROCESO POR FECHAS INDIVIDUALES (ventanas de 1 día) ===
# === LISTA DE FECHAS RECOMENDADAS (nubosidad ≤ 20%) ===
RECOMMENDED_DATES = [
    "2025-02-07","2025-02-10","2025-02-25","2025-02-27",
    "2025-03-02","2025-03-04","2025-03-07","2025-03-09","2025-03-12","2025-03-14",
    "2025-03-19","2025-03-22","2025-03-24","2025-03-26",
    "2025-04-03","2025-04-11","2025-04-13","2025-04-15","2025-04-16","2025-04-18",
    "2025-04-28","2025-05-03","2025-05-13","2025-05-28",
    "2025-07-10","2025-07-17","2025-07-20","2025-07-24","2025-08-01"
]

import pandas as pd

def day_window(d):
    d0 = pd.to_datetime(d)
    return [d0.strftime("%Y-%m-%d"), (d0 + pd.Timedelta(days=1)).strftime("%Y-%m-%d")]

# TIP: para procesar en tandas pequeñas, ajusta aquí
# DATES_BATCH = RECOMMENDED_DATES[:5]
DATES_BATCH = RECOMMENDED_DATES

conn = connect_openeo()
rows = []

for d in DATES_BATCH:
    win = day_window(d)

    # Atitlán
    try:
        rows.append(
            process_week_for_lake(
                conn,
                lake_name="Atitlán",
                aoi=LAKES["Atitlán"],
                date_window=win,
                scene_date=d,
                cloud_cover=20
            )
        )
        print(f"[OK] Atitlán {d}")
    except Exception as e:
        print(f"[SKIP] Atitlán {d} -> {e}")

    # Amatitlán
    try:
        rows.append(
            process_week_for_lake(
                conn,
                lake_name="Amatitlán",
                aoi=LAKES["Amatitlán"],
                date_window=win,
                scene_date=d,
                cloud_cover=20
            )
        )
        print(f"[OK] Amatitlán {d}")
    except Exception as e:
        print(f"[SKIP] Amatitlán {d} -> {e}")

# Append idempotente al CSV global
import numpy as np

df_new = pd.DataFrame(rows)
display(df_new)

df_out = append_timeseries(CSV_PATH, df_new)
print(f"Actualizado {CSV_PATH} → total filas: {len(df_out)}")

Authenticated using refresh token.
0:00:00 Job 'j-25081420390646c9b5d75aa6928c4fba': send 'start'
0:00:13 Job 'j-25081420390646c9b5d75aa6928c4fba': created (progress 0%)
0:00:19 Job 'j-25081420390646c9b5d75aa6928c4fba': created (progress 0%)
0:00:26 Job 'j-25081420390646c9b5d75aa6928c4fba': created (progress 0%)
0:00:34 Job 'j-25081420390646c9b5d75aa6928c4fba': created (progress 0%)
0:00:44 Job 'j-25081420390646c9b5d75aa6928c4fba': created (progress 0%)
0:00:57 Job 'j-25081420390646c9b5d75aa6928c4fba': created (progress 0%)
0:01:12 Job 'j-25081420390646c9b5d75aa6928c4fba': running (progress N/A)
0:01:32 Job 'j-25081420390646c9b5d75aa6928c4fba': running (progress N/A)
0:01:56 Job 'j-25081420390646c9b5d75aa6928c4fba': running (progress N/A)
0:02:26 Job 'j-25081420390646c9b5d75aa6928c4fba': finished (progress 100%)


/var/folders/kd/4y1c0b3j1273_pm_gb0hfhdw0000gn/T/ipykernel_25750/2224465159.py:27: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(str(out_dir))
/Users/gerco/UVG/8th_semester/Data_Science/DC-Lab4/venv/lib/python3.10/site-packages/openeo/rest/job.py:199: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
/Users/gerco/UVG/8th_semester/Data_Science/DC-Lab4/venv/lib/python3.10/site-packages/openeo/rest/job.py:203: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


[OK] Atitlán 2025-02-07
0:00:00 Job 'j-2508142044004acd855e435a3df06050': send 'start'
0:00:13 Job 'j-2508142044004acd855e435a3df06050': queued (progress 0%)
0:00:18 Job 'j-2508142044004acd855e435a3df06050': queued (progress 0%)
0:00:25 Job 'j-2508142044004acd855e435a3df06050': queued (progress 0%)
0:00:33 Job 'j-2508142044004acd855e435a3df06050': queued (progress 0%)
0:00:43 Job 'j-2508142044004acd855e435a3df06050': queued (progress 0%)
0:00:56 Job 'j-2508142044004acd855e435a3df06050': queued (progress 0%)
0:01:11 Job 'j-2508142044004acd855e435a3df06050': running (progress N/A)
0:01:31 Job 'j-2508142044004acd855e435a3df06050': running (progress N/A)
0:01:55 Job 'j-2508142044004acd855e435a3df06050': running (progress N/A)
0:02:25 Job 'j-2508142044004acd855e435a3df06050': finished (progress 100%)
[OK] Amatitlán 2025-02-07
0:00:00 Job 'j-25081420464543fa89b9e260041c2273': send 'start'
0:00:13 Job 'j-25081420464543fa89b9e260041c2273': created (progress 0%)
0:00:19 Job 'j-25081420464543fa8

,date,lake,chl_mean,chl_median,chl_p25,chl_p75
0,2025-02-07,Atitlán,42.474064,20.899792,10.655164,42.604095
1,2025-02-07,Amatitlán,111.009659,102.274704,86.056862,128.964249
2,2025-02-10,Atitlán,8.951129,7.584282,6.286437,10.782980
3,2025-02-25,Atitlán,16.665104,16.988651,14.829579,18.736156
4,2025-02-27,Atitlán,16.134600,15.959796,14.462414,17.727194
5,2025-02-27,Amatitlán,75.908188,80.717224,50.323441,102.759125
6,2025-03-02,Atitlán,19.695658,14.375379,9.183393,21.819719
7,2025-03-04,Atitlán,15.603611,15.709960,13.897983,17.441000
8,2025-03-04,Amatitlán,148.569138,135.697815,105.180649,187.255478
9,2025-03-07,Atitlán,27.255985,14.942939,8.907138,19.900459


Actualizado data/chl_timeseries.csv → total filas: 50
